#**Data Collection**
Scraping/Collecting data from Hockey Refrence (www.hockey-reference.com) by extracting the tables from the HTML using BeautifulSoup.

In [ ]:
import time
import requests
from io import StringIO
standings_url = "https://www.hockey-reference.com/leagues/NHL_2025.html"
data = requests.get(standings_url)
from bs4 import BeautifulSoup
soup = BeautifulSoup(data.text)
standings_table0 = soup.select("table.stats_table")[0] # find the table with the stats_table class
standings_table1 = soup.select("table.stats_table")[1] # find the table with the stats_table class
links0 = standings_table0.find_all("a") # find all of the a tags
links1 = standings_table1.find_all("a") # find all of the a tags
links = links0 + links1
links = [l.get("href") for l in links] # get the href value for each tag
modified_links = []
for link in links:
  modified_links.append(link.replace(".html", "_gamelog.html"))
team_urls = [f"https://www.hockey-reference.com{l}" for l in modified_links]
team_url = team_urls[0]
data = requests.get(team_url)
import pandas as pd
matches = pd.read_html(StringIO(data.text), match="Regular Season")
matches[0]

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                   Rk                Gtm               Date   
0                   1                  1         2024-10-09   
1                   2                  2         2024-10-10   
2                   3                  3         2024-10-12   
3                   4                  4         2024-10-16   
4                   5                  5         2024-10-19   
..                ...                ...                ...   
84                 79                 79         2025-04-12   
85                 80                 80         2025-04-13   
86                 81                 81         2025-04-15   
87                 82                 82         2025-04-17   
88                NaN                NaN                NaN   

   Unnamed: 3_level_0 Unnamed: 4_level_0    Score                 Team  ...  \
   Unnamed: 3_level_1                Opp     Rslt   GF   GA   OT   SOG  ...   
0                   @                MTL        L    0    1  NaN    48  ...   
1                   @                NJD        W    4    2  NaN    22  ...   
2                 NaN                PIT        W    4    2  NaN    33  ...   
3                 NaN                LAK        W    6    2  NaN    26  ...   
4                 NaN                NYR        L    1    4  NaN    35  ...   
..                ...                ...      ...  ...  ...  ...   ...  ...   
84                NaN                MTL        W    1    0   OT    35  ...   
85                  @                CAR        W    4    1  NaN    20  ...   
86                  @                BUF        W    4    0  NaN    32  ...   
87                NaN                DET        W    4    3   OT    20  ...   
88                NaN                NaN  52-26-4  268  231  NaN  2297  ...   

   Faceoffs       Advanced (5-on-5)                                             
        FOL   FO%                CF    CA   CF%    FF    FA   FF%  oZS%    PDO  
0        12  66.7                83    32  72.2    63    21  75.0  60.0   96.3  
1        28  48.1                37    48  43.5    24    27  47.1  44.0  112.4  
2        22  53.2                38    44  46.3    30    27  52.6  40.9  103.3  
3        27  55.0                27    59  31.4    20    45  30.8  33.3  117.6  
4        24  52.9                63    42  60.0    40    33  54.8  47.8   89.9  
..      ...   ...               ...   ...   ...   ...   ...   ...   ...    ...  
84       26  45.8                36    46  43.9    25    29  46.3  61.9  100.0  
85       23  56.6                35    52  40.2    31    36  46.3  34.5  117.2  
86       31  50.8                55    51  51.9    44    38  53.7  40.6  110.7  
87       16  64.4                38    44  46.3    25    34  42.4  43.5  103.7  
88     2094  53.7              3685  3976  48.1  2664  2867  48.2  47.6   99.9  

[89 rows x 30 columns]

In [ ]:
matches[0].columns = matches[0].columns.droplevel()
matches[0].head()

,Rk,Gtm,Date,Unnamed: 3_level_1,Opp,Rslt,GF,GA,OT,SOG,...,FOL,FO%,CF,CA,CF%,FF,FA,FF%,oZS%,PDO
0,1,1,2024-10-09,@,MTL,L,0,1,NaN,48,...,12,66.7,83,32,72.2,63,21,75.0,60.0,96.3
1,2,2,2024-10-10,@,NJD,W,4,2,NaN,22,...,28,48.1,37,48,43.5,24,27,47.1,44.0,112.4
2,3,3,2024-10-12,NaN,PIT,W,4,2,NaN,33,...,22,53.2,38,44,46.3,30,27,52.6,40.9,103.3
3,4,4,2024-10-16,NaN,LAK,W,6,2,NaN,26,...,27,55.0,27,59,31.4,20,45,30.8,33.3,117.6
4,5,5,2024-10-19,NaN,NYR,L,1,4,NaN,35,...,24,52.9,63,42,60.0,40,33,54.8,47.8,89.9


In [ ]:
years = list(range(2025, 2000, -1)) # All games from the 2000-01 season to the 2024-25 season
all_matches = [] # All
team_abbreviations = set() # Use a set to store unique team abbreviations

# Get all unique team abbreviations from one year
standings_url_2025 = "https://www.hockey-reference.com/leagues/NHL_2025.html"
data_2025 = requests.get(standings_url_2025)
soup_2025 = BeautifulSoup(data_2025.text)
standings_table0_2025 = soup_2025.select("table.stats_table")[0]
standings_table1_2025 = soup_2025.select("table.stats_table")[1]
links0_2025 = standings_table0_2025.find_all("a") # find all of the a tags
links1_2025 = standings_table1_2025.find_all("a") # find all of the a tags
links_2025 = links0_2025 + links1_2025
team_abbreviations.update([l.get("href").split("/")[-2] for l in links_2025 if l.get("href") and "/teams/" in l.get("href")]) # Extract abbreviations and add to set

# Now iterate through years and team abbreviations to get game logs
for year in years:
    # print(f"Getting data for year {year}") # Uncomment this line for debugging if there is an issue
    for team_abbreviation in team_abbreviations:
        if (year <= 2024 and team_abbreviation == "UTA"): # before 2024-25 UTA was called ARI
            team_abbreviation = "ARI"
        team_url = f"https://www.hockey-reference.com/teams/{team_abbreviation}/{year}_gamelog.html"
        # print(f"Attempting to get data for team {team_abbreviation} from {team_url}")

        try:
            data = requests.get(team_url)
            matches = pd.read_html(StringIO(data.text), match="Regular Season")[0]

            matches.columns = matches.columns.droplevel()

            matches["Season"] = year
            matches["Team"] = team_abbreviation
            all_matches.append(matches.reset_index(drop=True))
            # print(f"Successfully retrieved data for {team_abbreviation} in {year}") # Uncomment this line for debugging if there is an issue
        except ValueError:
            # print(f"Skipping {team_url} as no 'Regular Season' table was found.") # Uncomment this line for debugging if there is an issue
            pass

        time.sleep(2.5) # Space out the requests to avoid getting timed out

In [ ]:
len(all_matches)

704

In [ ]:
for i, df in enumerate(all_matches):
    if df.columns.duplicated().any():
        print(f"Problem in DataFrame {i}")
        print(df.columns.tolist())
        break

In [ ]:
def clean_scraped_df(df):
    # flatten multi-index columns if any
    if isinstance(df.columns[0], tuple):
        df.columns = ['_'.join([str(x) for x in col if x]) for col in df.columns]

    # strip whitespace
    df.columns = df.columns.str.strip()

    # auto-rename duplicate columns -> adds .1, .2 etc.
    cols = pd.Series(df.columns)
    for dup in cols[cols.duplicated()].unique():
        cols[cols[cols == dup].index.values.tolist()] = [dup + '.' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]
    df.columns = cols

    # now apply the hockey-specific mapping
    rename_map = {
        "SOG": "SOG_for",
        "PIM": "PIM_for",
        "PPG": "PPG_for",
        "PPO": "PPO_for",
        "SHG": "SHG_for",
        "SOG.1": "SOG_against",
        "PIM.1": "PIM_against",
        "PPG.1": "PPG_against",
        "PPO.1": "PPO_against",
        "SHG.1": "SHG_against",
    }
    df = df.rename(columns=rename_map)

    return df.reset_index(drop=True)

# clean all DataFrames
all_matches = [clean_scraped_df(df) for df in all_matches]
match_df = pd.concat(all_matches, ignore_index=True)

In [ ]:
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df.to_csv("matches.csv") # Save dataset copy to work with (raw, not yet cleaned)

#**Cleaning the Collected Data**
Cleaning the dataset by removing undesired rows and correcting team names.

In [ ]:
import pandas as pd

matches = pd.read_csv("matches.csv", index_col=0)

/tmp/ipython-input-4197675082.py:3: DtypeWarning: Columns (13,18,20,21,22,23,24,25,26,27,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  matches = pd.read_csv("matches.csv", index_col=0)


In [ ]:
matches["team"].value_counts()

,count
team,
STL,2059
EDM,2059
DET,2059
LAK,2059
OTT,2059
MTL,2059
CGY,2058
TBL,2058
BOS,2058


In [ ]:
# Data Cleaning / Fixing:
# Replace "ARI" with "UTAH" because the team relocated
matches['team'] = matches['team'].replace('ARI', 'UTA')
matches["opp"] = matches["opp"].replace('ARI', 'UTA')
display(matches['team'].value_counts())

# Remove rows that are headers (displaying the collumn's data type)
matches = matches[pd.to_numeric(matches['rk'], errors='coerce').notna()]
display(matches.head())
matches.shape



,count
team,
STL,2059
DET,2059
LAK,2059
OTT,2059
EDM,2059
MTL,2059
TBL,2058
BOS,2058
COL,2058


,rk,gtm,date,unnamed: 3_level_1,opp,rslt,gf,ga,ot,sog_for,...,cf,ca,cf%,ff,fa,ff%,ozs%,pdo,season,team
0,1,1,2024-10-09,NaN,CGY,L,5,6,OT,24,...,48,46,51.1,29,36,44.6,39.3,88.0,2025,VAN
1,2,2,2024-10-11,NaN,PHI,L,2,3,SO,26,...,48,34,58.5,30,25,54.5,37.9,106.9,2025,VAN
2,3,3,2024-10-15,@,TBL,L,1,4,NaN,27,...,49,48,50.5,29,37,43.9,50.0,85.2,2025,VAN
3,4,4,2024-10-17,@,FLA,W,3,2,OT,33,...,49,58,45.8,40,41,49.4,36.6,99.8,2025,VAN
4,5,5,2024-10-19,@,PHI,W,3,0,NaN,32,...,51,46,52.6,40,37,51.9,63.9,109.7,2025,VAN


(55558, 32)

In [ ]:
matches.dtypes

,0
rk,object
gtm,object
date,object
unnamed: 3_level_1,object
opp,object
rslt,object
gf,object
ga,object
ot,object
sog_for,object


In [ ]:
matches.to_csv("matches.csv") # Cleaned dataset